# Fruits fresh and rotten for classification

1. introducció

Utiltzare la llibreria Keras de tensorflow per el tractament de les dades ja que tinc carpetes amb imatges en comptes de un csv com estaba habituat. Tensorflow no esta Suportat en la versió de python 3.11 per lo que he tingut que pasar a 3.10 per si voleu executar el codi vosaltres.

In [1]:
#llibreries a utilitzar
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn import metrics,neighbors
import seaborn as sns
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
TRAIN_PATH = 'data/dataset/train/'
TEST_PATH = 'data/dataset/test/'

In [10]:
labels = []
for class_ in os.listdir(TRAIN_PATH):
    labels.append(class_)
NUM_LABELS = len(labels)

print(labels)

['freshapples', 'freshbanana', 'freshoranges', 'rottenapples', 'rottenbanana', 'rottenoranges']
